# Run in Collab

In [1]:
!git clone https://github.com/time-series-foundation-models/lag-llama/

Cloning into 'lag-llama'...
remote: Enumerating objects: 468, done.
remote: Counting objects: 100% (304/304), done.
remote: Compressing objects: 100% (170/170), done.
remote: Total 468 (delta 194), reused 212 (delta 130), pack-reused 164 (from 1)
Receiving objects: 100% (468/468), 280.63 KiB | 8.77 MiB/s, done.
Resolving deltas: 100% (236/236), done.


In [2]:
cd /content/lag-llama

/content/lag-llama


In [3]:
!pip install -r requirements.txt --quiet # this could take some time # ignore the errors displayed by colab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 106.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.1/778.1 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.4/891.4 kB 52.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.1.4 which is incompatible.
mizani 0.13.0 requires pandas>=2.2.0, but you have pandas 2.1.4 which is incompatible.
plotnine 0.14.1 requires pandas>=2.2.0, but you have pandas 2.1.4 which is incompatible.


download pretrained model weights from [HuggingFace](https://huggingface.co/time-series-foundation-models/Lag-Llama) 🤗

In [4]:
!huggingface-cli download time-series-foundation-models/Lag-Llama lag-llama.ckpt --local-dir /content/lag-llama

lag-llama.ckpt: 100% 29.5M/29.5M [00:00<00:00, 40.1MB/s]
Download complete. Moving file to /content/lag-llama/lag-llama.ckpt
/content/lag-llama/lag-llama.ckpt


We import the required packages and the lag llama estimator object which we can use to make predictions.

In [5]:
from itertools import islice

from matplotlib import pyplot as plt
import matplotlib.dates as mdates

import torch
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.dataset.repository.datasets import get_dataset

from gluonts.dataset.pandas import PandasDataset
import pandas as pd

from lag_llama.gluon.estimator import LagLlamaEstimator

# Lag-Llama prediction function

We create a function for Lag-Llama inference that we can reuse for all different types of dataset below. This function returns the predictions for the given prediction horizon. The forecast will be of shape (`num_samples`, `prediction_length`), where `num_samples` is the number of samples sampled from the predicted probability distribution for each timestep.


In [6]:
def get_lag_llama_predictions(dataset, prediction_length, device, num_samples=100):
    ckpt = torch.load("lag-llama.ckpt", map_location=device) # Uses GPU since in this Colab we use a GPU.
    estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

    estimator = LagLlamaEstimator(
        ckpt_path="lag-llama.ckpt",
        prediction_length=prediction_length,
        context_length=32, # Should not be changed; this is what the released Lag-Llama model was trained with

        # estimator args
        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        scaling=estimator_args["scaling"],
        time_feat=estimator_args["time_feat"],

        batch_size=1,
        num_parallel_samples=100,
        device=device,
    )

    lightning_module = estimator.create_lightning_module()
    transformation = estimator.create_transformation()
    predictor = estimator.create_predictor(transformation, lightning_module)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset=dataset,
        predictor=predictor,
        num_samples=num_samples
    )
    forecasts = list(forecast_it)
    tss = list(ts_it)

    return forecasts, tss

In [7]:
import pandas as pd
from gluonts.dataset.pandas import PandasDataset

url = (
    "https://gist.githubusercontent.com/rsnirwan/a8b424085c9f44ef2598da74ce43e7a3"
    "/raw/b6fdef21fe1f654787fa0493846c546b7f9c4df2/ts_long.csv"
)
df = pd.read_csv(url, index_col=0, parse_dates=True)
df

,target,item_id
2021-01-01 00:00:00,-1.3378,A
2021-01-01 01:00:00,-1.6111,A
2021-01-01 02:00:00,-1.9259,A
2021-01-01 03:00:00,-1.9184,A
2021-01-01 04:00:00,-1.9168,A
...,...,...
2021-01-10 19:00:00,1.2349,J
2021-01-10 20:00:00,1.1525,J
2021-01-10 21:00:00,1.1485,J
2021-01-10 22:00:00,1.3248,J


In [13]:
def simple_comparison_predictions(num_predictions, num_original):
    similar_list = []
    for i in range(len(num_predictions)):
        similar_list.append(num_predictions[i] == num_original[i])
    return 1 if all(similar_list) else 0

def ask_pred_by_last_digits(one_string_sequence, num_digits_to_predict):
    str_list = one_string_sequence.split(",")
    num_list = [int(numeric_string) for numeric_string in str_list]
    # print(num_list)
    num_len = len(num_list)

    # Ensure we don't try to predict more digits than exist
    len_predictions = min(num_digits_to_predict, num_len)

    to_be_predicted = num_list[-len_predictions:]
    root = num_list[: num_len - len_predictions]

    # print("num_len: ", num_len)
    # print("len_predictions: ", len_predictions)
    # print("root: ", root)
    # print("to_be_predicted: ", to_be_predicted)
    return root, to_be_predicted

def predict_last_digit_over_list_sequences(list_str_seq, num_last_digits=1):
    counter = 0

    roots_list = []
    expected_list = []
    predicted_list = []
    success_list = []


    for s in list_str_seq:
      counter += 1

      r, e = ask_pred_by_last_digits(s, num_last_digits)

      print("====== ", str(counter), " of ", str(len(list_str_seq)))
      # print("root: ", r)
      # print("expected: ", str(e))

      temp_df = pd.DataFrame()
      temp_df['values']= r
      ids = ['A'] * len(temp_df)
      temp_df['item_id'] = ids


      try:
        dataset = PandasDataset.from_long_dataframe(temp_df, target="values", item_id="item_id")
        backtest_dataset = dataset
        prediction_length = 1  # Define your prediction length. We use 24 here since the data is of hourly frequency
        num_samples = len(e)#len(temp_df) # number of samples sampled from the probability distribution for each timestep
        forecasts, tss = get_lag_llama_predictions(backtest_dataset, prediction_length, device, num_samples)
        forecasts = (forecasts[0].samples)[0:len(e)]
        # forecasts = forecasts.np()
        flat_list = [x for xs in forecasts for x in xs]
        forecasting = [int(round(x)) for x in flat_list]
      except:
        print("error in sequence: ", s)
        forecasting = [0] * len(e)

      predicted_list.append(forecasting)

      roots_list.append(r)
      expected_list.append(e)
      success_list.append(simple_comparison_predictions(forecasting, e))

      # print("forecast: ", forecasting)
      # print("tss: ", tss)
      # print("shapes: ", forecasts[0].samples.shape)
      # return forecast, tss
    return roots_list, expected_list, predicted_list, success_list

def create_results_df_4_digits(list_str_seq, num_last_digits=1, pred_algo="lag-llama"):
    pred_algo_list = [pred_algo] * len(list_str_seq)
    results_df = pd.DataFrame()
    roots, expecteds, predicteds, success_list = predict_last_digit_over_list_sequences(list_str_seq=list_str_seq, num_last_digits=1)

    # results_df["seq"] = list_str_seq
    results_df["forecasting_method_name"] = pred_algo_list
    results_df["root"] = roots
    results_df["to_be_predicted"] = expecteds
    results_df["forecasted"] = predicteds
    results_df["success"] = success_list

    return results_df

In [8]:
import math
import numpy as np

In [9]:
def create_original_length_test_pool():
    original_l1 = [
        "2, 4, 6, 8, 10, 12, 14, 16, 18, 20",
        "3, 6, 9, 12, 15, 18, 21, 24, 27, 30",
        "4, 8, 12, 16, 20, 24, 28, 32, 36, 40",
        "5, 10, 15, 20, 25, 30, 35, 40, 45, 50",
        "6, 12, 18, 24, 30, 36, 42, 48, 54, 60",
        "7, 14, 21, 28, 35, 42, 49, 56, 63, 70",
        "8, 16, 24, 32, 40, 48, 56, 64, 72, 80",
        "9, 18, 27, 36, 45, 54, 63, 72, 81, 90",
        "10, 20, 30, 40, 50, 60, 70, 80, 90, 100",
        "1, 3, 5, 7, 9, 11, 13, 15, 17, 19",
        "2, 4, 6, 8, 10, 12, 14, 16, 18, 20",
        "11, 12, 13, 14, 15, 16, 17, 18, 19, 20",
        "21, 22, 23, 24, 25, 26, 27, 28, 29, 30",
        "31, 32, 33, 34, 35, 36, 37, 38, 39, 40",
        "41, 42, 43, 44, 45, 46, 47, 48, 49, 50",
        "51, 52, 53, 54, 55, 56, 57, 58, 59, 60",
        "61, 62, 63, 64, 65, 66, 67, 68, 69, 70",
        "71, 72, 73, 74, 75, 76, 77, 78, 79, 80",
        "81, 82, 83, 84, 85, 86, 87, 88, 89, 90",
        "91, 92, 93, 94, 95, 96, 97, 98, 99, 100",
        "101, 102, 103, 104, 105, 106, 107, 108, 109, 110",
        "111, 112, 113, 114, 115, 116, 117, 118, 119, 120",
        "121, 122, 123, 124, 125, 126, 127, 128, 129, 130",
        "131, 132, 133, 134, 135, 136, 137, 138, 139, 140",
        "141, 142, 143, 144, 145, 146, 147, 148, 149, 150",
        "151, 152, 153, 154, 155, 156, 157, 158, 159, 160",
        "161, 162, 163, 164, 165, 166, 167, 168, 169, 170",
        "171, 172, 173, 174, 175, 176, 177, 178, 179, 180",
        "181, 182, 183, 184, 185, 186, 187, 188, 189, 190",
        "191, 192, 193, 194, 195, 196, 197, 198, 199, 200",
    ]

    original_l2 = [
        "2, 3, 5, 7, 11, 13, 17, 19, 23, 29",
        "1, 1, 2, 3, 5, 8, 13, 21, 34, 55",
        "1, 2, 4, 8, 16, 32, 64, 128, 256, 512",
        "1, 3, 9, 27, 81, 243, 729, 2187, 6561, 19683",
        "1, 4, 9, 16, 25, 36, 49, 64, 81, 100",
        "1, 8, 27, 64, 125, 216, 343, 512, 729, 1000",
        "1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880",
        "1, 3, 6, 10, 15, 21, 28, 36, 45, 55",
        "2, 1, 3, 4, 7, 11, 18, 29, 47, 76",
        "0, 1, 2, 5, 12, 29, 70, 169, 408, 985",
        "1, 4, 27, 256, 3125, 46656, 823543, 16777216, 387420489, 10000000000",
        "1, 2, 6, 20, 70, 252, 924, 3432, 12870, 48620",
        "2, 3, 5, 7, 11, 13, 17, 19, 23, 29",
        "4, 6, 9, 10, 14, 15, 21, 22, 25, 26",
        "1, 10, 11, 100, 101, 110, 111, 1000, 1001, 1010",
        "0, 1, 81, 512, 2401, 4913, 5832, 17576, 19683, 234256",
        "1, 2, 145, 40585",
        "2, 5, 12, 20, 29, 39, 50, 62, 75, 89",
        "1, 8, 10, 18, 19, 100, 101, 108, 109, 110",
        "3, 7, 31, 127, 2047, 8191, 131071, 524287, 8388607, 536870911",
        "1, 2, 4, 8, 16, 23, 28, 38, 58, 89",
        "1, 2, 4, 8, 15, 26, 42, 64, 93, 129",
        "1, 5, 12, 22, 35, 51, 70, 92, 117, 145",
        "0, 1, 1, 2, 1, 2, 2, 3, 1, 3",
        "1, 2, 5, 15, 52, 203, 877, 4140, 21147, 115975",
        "2, 3, 5, 7, 11, 13, 17, 19, 23, 29",
        "1, 11, 21, 1211, 111221",
        "2, 3, 5, 7, 11, 13, 17, 19, 23, 29",
        "1, 2, 4, 8, 16, 32, 64, 128, 256, 512",
        "1, 3, 7, 15, 31, 63, 127, 255, 511, 1023",
    ]

    original_l3 = [
        "1, 4, 9, 16, 25, 36, 49, 64, 81, 100",
        "1, 8, 27, 64, 125, 216, 343, 512, 729, 1000",
        "2, 1, 2, 1, 2, 2, 1, 2, 2, 2",
        "1, 2, 2, 3, 3, 4, 4, 4, 5, 5",
        "2, 4, 8, 16, 32, 64, 128, 256, 512, 1024",
        "2, 5, 12, 20, 29, 39, 50, 62, 75, 89",
        "7, 11, 13, 17, 19, 23, 29, 31, 37, 41",
        "1, 11, 111, 1111, 11111, 111111, 1111111, 11111111, 111111111, 1111111111",
        "31, 331, 3331, 33331, 333331, 3333331, 33333331, 333333331",
        "1, 3, 7, 13, 21, 31, 43, 57, 73, 91",
        "6, 28, 496, 8128, 130816, 2096128",
        "0, 0, 2, 3, 5, 7, 11, 15, 22, 30",
        "3, 37, 67, 101, 137, 199, 269, 307, 353, 389",
        "1, 2, 145, 40585",
        "19, 31, 61, 89, 107, 127, 521, 607, 1279, 2203",
        "2, 3, 7, 42, 1806, 3263442",
        "2, 3, 5, 7, 11, 13, 17, 19, 23, 29",
        "1, 8, 10, 18, 19, 100, 101, 108, 109, 110",
        "0, 1, 1, 2, 1, 2, 2, 3, 1, 3",
        "0, 1, 1, 2, 2, 4, 2, 6, 4, 6",
        "2, 3, 5, 7, 11, 13, 17, 19, 23, 29",
        "1, 4, 2, 1, 3, 5, 9, 2, 6, 5",
        "1, 7, 3, 2, 0, 5, 0, 7, 9, 8",
        "1, 0, 1, 2, 3, 1, 6, 11",
        "1, 1, 1, 3, 7, 15, 31, 63",
        "3, 5, 11, 17, 31, 41, 59, 67",
        "1, 2, 3, 4, 6, 8, 11, 13",
        "1, 0, 1, 1, 2, 3, 6, 11",
        "1093, 3511",
        "5, 13, 563",
    ]
    original_l4 = [
        "29, 57, 68, 120, 134, 140, 173, 197, 283, 313",
        "24, 26, 36, 40, 184, 226, 244, 384, 391, 423",
        "90, 203, 212, 235, 270, 324, 342, 352, 371, 417",
        "20, 48, 95, 234, 282, 296, 352, 402, 428, 481",
        "62, 98, 130, 154, 290, 315, 324, 385, 408, 447",
        "2, 42, 66, 102, 153, 195, 201, 252, 306, 396",
        "128, 151, 153, 217, 224, 332, 382, 400, 450, 478",
        "26, 50, 114, 148, 160, 170, 274, 347, 432, 497",
        "48, 94, 176, 177, 219, 276, 282, 283, 459, 488",
        "139, 252, 272, 281, 304, 361, 370, 415, 438, 500",
        "15, 95, 115, 195, 240, 318, 326, 350, 432, 450",
        "134, 224, 293, 378, 379, 395, 434, 451, 482, 496",
        "23, 93, 142, 145, 245, 266, 296, 317, 428, 495",
        "18, 39, 71, 194, 197, 219, 263, 270, 416, 473",
        "9, 84, 144, 170, 325, 393, 401, 405, 435, 497",
        "26, 40, 202, 267, 282, 340, 359, 408, 410, 495",
        "34, 92, 164, 165, 209, 296, 414, 456, 467, 494",
        "16, 119, 121, 123, 135, 139, 285, 311, 409, 412",
        "8, 11, 12, 103, 116, 196, 247, 254, 389, 427",
        "12, 36, 96, 119, 171, 213, 221, 232, 363, 451",
        "38, 91, 142, 197, 215, 313, 316, 319, 423, 466",
        "7, 42, 147, 201, 213, 248, 310, 332, 436, 479",
        "27, 101, 105, 164, 245, 290, 304, 441, 449, 490",
        "4, 11, 29, 106, 214, 283, 296, 298, 360, 497",
        "72, 106, 139, 165, 171, 192, 199, 429, 453, 477",
        "187, 218, 260, 295, 301, 314, 379, 410, 452, 469",
        "29, 63, 95, 140, 150, 190, 221, 437, 482, 491",
        "3, 11, 84, 144, 156, 177, 188, 199, 229, 284",
        "26, 94, 98, 137, 176, 301, 323, 330, 372, 444",
        "39, 81, 88, 210, 215, 378, 416, 430, 439, 490",
    ]

    return [original_l1, original_l2, original_l3, original_l4]

def create_simple_bin_test_pool():
    return [
    "0,0,0,0,0,0,0,0",
    "0,0,0,0,0,0,0,1",
    "0,0,0,0,1,0,0,0",
    "0,0,0,0,0,1,0,0,0",
    "0,0,0,0,0,0,1,1,0",
    "0,0,0,0,0,0,0,1,0",
    "0,0,0,0,0,0,0,1,1",
    "0,0,0,1,1,0,0,0,0",
    "0,0,1,0,0,0,0,0,1",
    "0,1,0,1,0,1,0,1,0",
    "0,0,0,0,1,1,1,0,1",
    "0,0,0,0,0,0,0,1,0,0",
    "0,0,0,0,0,0,0,0,1,0",
    "0,0,0,0,0,0,0,0,1,0",
    "0,0,0,0,0,1,1,0,1,0",
    "0,1,0,1,0,1,0,1,0,1",
    "0,0,1,0,1,0,1,0,1,0",
    "0,1,1,0,1,1,0,1,1,0",
    "0,0,0,0,0,0,0,0,0,1",
    "0,0,1,0,1,0,1,1,0,1",
    "0,1,0,1,0,0,1,0,0,1",
    "0,1,0,1,0,1,1,0,1,0",
    "0,0,0,1,0,1,0,1,0,1,0",
    "0,1,0,1,0,1,0,1,0,1,0"
    ]

def create_ramdom_bin_test_pool():
    return[
        "1,1,0,0,0,1,0,0,0,0,1,0",
        "0,0,0,1,0,1,1,0,1,0,1,1",
        "1,0,0,0,0,1,1,1,1,0,0,1",
        "1,1,1,1,1,1,1,1,0,1,0,0",
        "1,0,1,1,0,1,0,1,0,1,0,0",
        "1,1,1,0,0,1,1,0,1,1,0,1",
        "0,0,1,1,1,0,0,1,0,1,1,0",
        "1,1,1,0,0,0,0,0,0,0,0,0",
        "0,1,1,1,1,0,1,0,1,1,1,1",
        "1,1,0,0,0,1,1,1,0,1,1,1",
        "1,0,0,1,1,0,1,0,0,0,1,0",
        "0,1,1,1,0,1,1,0,1,1,0,1",
        "1,0,1,0,0,1,0,1,0,1,1,1",
        "0,1,1,0,0,1,1,1,1,0,0,1",
        "0,0,1,1,1,1,1,0,1,1,1,0",
        "1,1,1,0,1,1,1,0,0,1,0,0",
        "1,0,1,0,1,0,0,0,0,0,0,0",
        "0,0,1,1,1,1,1,1,0,1,1,1",
        "1,1,0,0,0,1,0,1,0,1,0,1",
        "0,0,0,0,0,1,0,1,1,0,1,0",
        "0,0,1,1,1,1,1,1,0,0,1,1",
        "0,1,0,0,1,0,0,1,0,1,1,1",
        "1,0,0,1,1,0,0,0,0,0,0,1",
        "1,1,0,1,1,1,0,0,1,1,1,0",
        "0,0,0,0,0,0,0,1,0,0,0,0",
        "0,1,0,0,1,1,1,1,1,1,0,1",
        "0,0,1,0,0,0,1,0,0,0,0,0",
        "1,0,0,0,1,1,0,0,1,1,0,0",
        "1,1,0,0,1,0,0,0,0,0,0,0",
        "1,0,1,0,1,0,1,0,0,1,1,1",
        "1,0,1,1,0,1,1,1,1,1,0,0",
        "0,0,0,0,0,0,1,0,0,0,0,0",
        "0,1,1,0,0,0,0,0,0,0,1,1",
        "0,0,1,1,0,1,1,0,1,1,0,1",
        "0,0,0,0,1,1,1,1,1,0,1,1",
        "1,1,1,0,0,0,0,1,1,1,1,1",
        "1,0,1,0,0,1,1,1,1,1,1,1",
        "0,1,0,0,0,0,0,0,1,0,1,0",
        "0,1,0,0,0,1,0,1,1,1,0,0",
        "1,0,1,0,1,1,1,0,1,0,1,0",
        "1,0,0,0,0,1,0,0,1,1,1,1",
        "1,1,0,0,0,0,1,0,0,0,1,1",
        "1,0,1,0,0,0,1,0,1,0,0,0",
        "0,0,1,1,1,1,0,0,1,0,1,0",
        "0,0,0,0,0,0,1,0,0,0,1,0",
        "1,1,1,1,0,0,1,1,1,0,0,1",
        "1,0,1,0,1,1,0,1,0,1,1,0",
        "1,0,0,1,0,1,1,1,1,1,1,0",
        "1,1,1,1,1,0,0,0,1,1,1,0",
        "0,0,0,1,0,1,1,1,0,0,1,1",
        "0,0,1,0,1,0,0,1,0,0,0,0",
        "1,1,1,0,0,1,1,1,0,1,1,1",
        "1,0,0,1,1,1,0,1,1,1,1,1",
        "1,0,0,1,0,0,1,1,0,0,1,1",
        "0,0,0,0,0,0,1,1,0,1,1,1",
        "1,0,0,1,1,0,0,1,1,1,1,0",
        "1,1,1,1,1,0,0,1,0,0,0,1",
        "0,1,1,1,0,1,1,1,1,1,1,1",
        "1,0,1,0,0,1,0,0,1,1,0,1",
        "0,1,0,0,0,1,1,0,0,1,1,1",
        "0,0,0,0,0,1,1,0,1,1,0,1",
        "0,1,0,0,1,0,0,1,0,0,1,0",
        "1,1,1,0,0,1,1,1,1,1,0,1",
        "0,1,0,1,0,1,0,0,1,1,0,1",
        "1,1,1,0,1,0,1,0,0,0,0,1",
        "1,0,0,0,1,0,1,1,1,1,0,1",
        "1,0,1,0,0,0,1,0,1,1,0,1",
        "0,0,1,0,0,1,0,1,1,0,1,1",
        "1,0,0,1,0,1,1,0,0,1,1,0",
        "1,1,0,0,1,1,1,0,0,1,0,1",
        "0,0,1,0,1,1,1,0,0,1,1,1",
        "0,1,1,1,0,1,1,0,1,1,1,0",
        "1,0,0,0,0,1,0,0,1,0,1,0",
        "0,1,1,1,1,0,0,1,0,0,0,0",
        "0,0,0,0,0,1,1,1,0,1,0,0",
        "0,1,0,1,0,1,1,0,0,1,1,0",
        "0,0,0,1,0,0,0,0,1,0,1,0",
        "0,1,1,0,0,0,0,0,1,1,0,1",
        "1,0,0,1,0,1,0,1,1,0,0,0",
        "0,1,0,1,1,1,1,1,0,1,0,1",
        "1,0,0,1,0,0,1,0,1,1,0,0",
        "0,1,0,1,0,1,0,1,1,1,0,1",
        "0,0,1,0,0,1,1,0,1,0,0,1",
        "1,1,0,0,0,0,1,1,1,1,1,0",
        "1,0,1,1,0,0,0,1,0,0,1,0",
        "0,1,1,1,0,0,1,1,0,0,0,1",
        "1,0,1,1,1,0,1,0,1,0,1,0",
        "0,1,0,1,0,0,0,0,0,0,1,0",
        "0,1,0,0,0,1,1,0,1,0,0,0",
        "0,0,1,0,1,0,0,0,1,1,1,1",
        "1,0,1,1,1,0,1,1,1,0,1,1",
        "1,1,1,1,0,1,0,1,0,0,1,0",
        "0,0,1,1,0,0,1,0,1,0,1,0",
        "0,1,1,0,1,0,1,0,1,1,1,0",
        "1,0,1,0,1,0,1,1,0,0,1,0",
        "1,0,1,1,1,1,1,0,1,1,1,0",
        "0,1,0,1,0,0,1,1,0,1,0,1",
        "0,0,0,1,0,1,0,0,1,1,1,1",
        "1,1,1,0,0,0,1,1,1,1,1,1",
        "1,0,0,0,1,1,1,0,1,1,0,1"
    ]

def simple_comparison_predictions(num_predictions, num_original):
    similar_list = []
    for i in range(len(num_predictions)):
        similar_list.append(num_predictions[i] == num_original[i])
    return 1 if all(similar_list) else 0


def compare_predictions(num_predictions, num_original):

    # sort similarity percentage
    similar_list = []
    for i in range(len(num_predictions)):
        similar_list.append(num_predictions[i] == num_original[i])

    sort_similarity = (
        len([ele for ele in similar_list if ele is True]) / len(similar_list)
    ) * 100
    # General similarity percentage
    common_elements = set(num_predictions).intersection(set(num_original))
    num_common_elements = len(common_elements)

    # Find the total number of unique elements in both lists
    total_elements = set(num_predictions).union(set(num_original))
    num_total_elements = len(total_elements)

    # Calculate the percentage similarity
    general_similarity = (num_common_elements / num_total_elements) * 100

    # LEVENSHTEIN
    # pred_str = num_list_to_string(num_predictions)
    # orig_str = num_list_to_string(num_original)
    # print(pred_str, " ", orig_str)
    # lvtn = levenshtein(pred_str, orig_str)
    return sort_similarity, general_similarity #, lvtn


def ask_pred_by_percentage(one_string_sequence, final_percentage_to_predict):
    str_list = one_string_sequence.split(",")
    num_list = [int(numeric_string) for numeric_string in str_list]
    # print(num_list)
    num_len = len(num_list)
    len_predictions = math.floor((num_len * final_percentage_to_predict) / 100)

    to_be_predicted = num_list[-len_predictions:]
    root = num_list[: num_len - len_predictions]

    # print("num_len: ", num_len)
    # print(len_predictions)
    # print("root: ", root)
    # print("to_be_predicted: ", to_be_predicted)
    return root, to_be_predicted

percentages_list = [10, 25, 50, 75]
device = torch.device("cuda:0") # You can switch this to CPU or other GPUs if you'd like, depending on your environment

def predict_over_list_sequences(list_str_seq, percentage):
    counter = 0

    roots_list = []
    expected_list = []
    predicted_list = []
    general_similarity_list = []
    sort_similarity_list = []


    for s in list_str_seq:
      counter += 1

      r, e = ask_pred_by_percentage(s, percentage)
      roots_list.append(r)
      expected_list.append(e)
      print("====== ", str(counter), " of ", str(len(list_str_seq)))
      # print("root: ", r)
      # print("expected: ", str(e))

      temp_df = pd.DataFrame()
      temp_df['values']= r
      ids = ['A'] * len(temp_df)
      temp_df['item_id'] = ids


      try:
        dataset = PandasDataset.from_long_dataframe(temp_df, target="values", item_id="item_id")
        backtest_dataset = dataset
        prediction_length = 1  # Define your prediction length. We use 24 here since the data is of hourly frequency
        num_samples = len(e)#len(temp_df) # number of samples sampled from the probability distribution for each timestep
        forecasts, tss = get_lag_llama_predictions(backtest_dataset, prediction_length, device, num_samples)
        forecasts = (forecasts[0].samples)[0:len(e)]
        # forecasts = forecasts.np()
        flat_list = [x for xs in forecasts for x in xs]
        forecasting = [int(round(x)) for x in flat_list]
      except:
        print("error in sequence: ", s)
        forecasting = [0] * len(e)

      predicted_list.append(forecasting)

      sort_simi, gral_simi = compare_predictions(forecasting, e)
      sort_similarity_list.append(sort_simi)
      general_similarity_list.append(gral_simi)
      # print("forecast: ", forecasting)
      # print("tss: ", tss)
      # print("shapes: ", forecasts[0].samples.shape)
      # return forecast, tss
    return roots_list, expected_list, predicted_list, sort_similarity_list, general_similarity_list


def create_results_df(list_str_seq, complexity, percentage, pred_algo="lag-llama", seq_type="original"):
    complexity_list = [complexity] * len(list_str_seq)
    pred_algo_list = [pred_algo] * len(list_str_seq)
    seq_type_list = [seq_type] * len(list_str_seq)
    results_df = pd.DataFrame()
    roots, expecteds, predicteds, sort_simi, gral_simi = predict_over_list_sequences(list_str_seq=list_str_seq, percentage=percentage)

    # results_df["seq"] = list_str_seq
    results_df["complexity"] = complexity_list
    results_df["forecasting_method_name"] = pred_algo_list
    results_df["root_" + str(percentage)] = roots
    results_df["to_predict_" + str(percentage)] = expecteds
    results_df["forecasted_" + str(percentage)] = predicteds
    results_df["sort_simi_percen_" + str(percentage)] = sort_simi
    results_df["gral_simi_percen_" + str(percentage)] = gral_simi
    return results_df

def create_results_by_list_percentages(list_str_seq, complexity, percentages_list, pred_algo="lag-llama", seq_type="original"):
    general_df = pd.DataFrame()
    for per in percentages_list:
      r_df = create_results_df(list_str_seq, complexity, per, pred_algo, seq_type)
      general_df = pd.concat([general_df, r_df], axis=1)
    general_df["seq"] = list_str_seq
    return general_df




In [10]:
# Set numerical columns as float32
for col in df.columns:
    # Check if column is not of string type
    if df[col].dtype != 'object' and pd.api.types.is_string_dtype(df[col]) == False:
        df[col] = df[col].astype('float32')

# Create the Pandas
dataset = PandasDataset.from_long_dataframe(df, target="target", item_id="item_id")

backtest_dataset = dataset
prediction_length = 24  # Define your prediction length. We use 24 here since the data is of hourly frequency
num_samples = 100 # number of samples sampled from the probability distribution for each timestep
device = torch.device("cuda:0") # You can switch this to CPU or other GPUs if you'd like, depending on your environment

In [11]:
bin_r_seq = create_ramdom_bin_test_pool()
bin_r_seq

['1,1,0,0,0,1,0,0,0,0,1,0',
 '0,0,0,1,0,1,1,0,1,0,1,1',
 '1,0,0,0,0,1,1,1,1,0,0,1',
 '1,1,1,1,1,1,1,1,0,1,0,0',
 '1,0,1,1,0,1,0,1,0,1,0,0',
 '1,1,1,0,0,1,1,0,1,1,0,1',
 '0,0,1,1,1,0,0,1,0,1,1,0',
 '1,1,1,0,0,0,0,0,0,0,0,0',
 '0,1,1,1,1,0,1,0,1,1,1,1',
 '1,1,0,0,0,1,1,1,0,1,1,1',
 '1,0,0,1,1,0,1,0,0,0,1,0',
 '0,1,1,1,0,1,1,0,1,1,0,1',
 '1,0,1,0,0,1,0,1,0,1,1,1',
 '0,1,1,0,0,1,1,1,1,0,0,1',
 '0,0,1,1,1,1,1,0,1,1,1,0',
 '1,1,1,0,1,1,1,0,0,1,0,0',
 '1,0,1,0,1,0,0,0,0,0,0,0',
 '0,0,1,1,1,1,1,1,0,1,1,1',
 '1,1,0,0,0,1,0,1,0,1,0,1',
 '0,0,0,0,0,1,0,1,1,0,1,0',
 '0,0,1,1,1,1,1,1,0,0,1,1',
 '0,1,0,0,1,0,0,1,0,1,1,1',
 '1,0,0,1,1,0,0,0,0,0,0,1',
 '1,1,0,1,1,1,0,0,1,1,1,0',
 '0,0,0,0,0,0,0,1,0,0,0,0',
 '0,1,0,0,1,1,1,1,1,1,0,1',
 '0,0,1,0,0,0,1,0,0,0,0,0',
 '1,0,0,0,1,1,0,0,1,1,0,0',
 '1,1,0,0,1,0,0,0,0,0,0,0',
 '1,0,1,0,1,0,1,0,0,1,1,1',
 '1,0,1,1,0,1,1,1,1,1,0,0',
 '0,0,0,0,0,0,1,0,0,0,0,0',
 '0,1,1,0,0,0,0,0,0,0,1,1',
 '0,0,1,1,0,1,1,0,1,1,0,1',
 '0,0,0,0,1,1,1,1,1,0,1,1',
 '1,1,1,0,0,0,0,1,1,

In [14]:
res = create_results_df_4_digits(bin_r_seq)
res

======  1  of  100


<ipython-input-6-9be98728e089>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("lag-llama.ckpt", map_location=device) # Uses GPU since in this Colab we us

======  2  of  100


<ipython-input-6-9be98728e089>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("lag-llama.ckpt", map_location=device) # Uses GPU since in this Colab we us

======  3  of  100


...

======  99  of  100


<ipython-input-6-9be98728e089>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("lag-llama.ckpt", map_location=device) # Uses GPU since in this Colab we us

======  100  of  100


<ipython-input-6-9be98728e089>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("lag-llama.ckpt", map_location=device) # Uses GPU since in this Colab we us

,forecasting_method_name,root,to_be_predicted,forecasted,success
0,lag-llama,"[1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]",[0],[0],1
1,lag-llama,"[0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1]",[1],[0],0
2,lag-llama,"[1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]",[1],[0],0
3,lag-llama,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0]",[0],[1],0
4,lag-llama,"[1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0]",[0],[0],1
...,...,...,...,...,...
95,lag-llama,"[1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1]",[0],[1],0
96,lag-llama,"[0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0]",[1],[1],1
97,lag-llama,"[0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1]",[1],[0],0
98,lag-llama,"[1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1]",[1],[1],1


In [15]:
res.to_csv("bin_seq_last_digit_pred_lag_llama_random.csv", index=False)

In [ ]:
bin_seq = create_simple_bin_test_pool()
bin_seq

['0,0,0,0,0,0,0,0',
 '0,0,0,0,0,0,0,1',
 '0,0,0,0,1,0,0,0',
 '0,0,0,0,0,1,0,0,0',
 '0,0,0,0,0,0,1,1,0',
 '0,0,0,0,0,0,0,1,0',
 '0,0,0,0,0,0,0,1,1',
 '0,0,0,1,1,0,0,0,0',
 '0,0,1,0,0,0,0,0,1',
 '0,1,0,1,0,1,0,1,0',
 '0,0,0,0,1,1,1,0,1',
 '0,0,0,0,0,0,0,1,0,0',
 '0,0,0,0,0,0,0,0,1,0',
 '0,0,0,0,0,0,0,0,1,0',
 '0,0,0,0,0,1,1,0,1,0',
 '0,1,0,1,0,1,0,1,0,1',
 '0,0,1,0,1,0,1,0,1,0',
 '0,1,1,0,1,1,0,1,1,0',
 '0,0,0,0,0,0,0,0,0,1',
 '0,0,1,0,1,0,1,1,0,1',
 '0,1,0,1,0,0,1,0,0,1',
 '0,1,0,1,0,1,1,0,1,0',
 '0,0,0,1,0,1,0,1,0,1,0',
 '0,1,0,1,0,1,0,1,0,1,0']

In [ ]:
res = create_results_df_4_digits(bin_seq)
res

======  1  of  24


<ipython-input-6-9be98728e089>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("lag-llama.ckpt", map_location=device) # Uses GPU since in this Colab we us

======  2  of  24


<ipython-input-6-9be98728e089>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("lag-llama.ckpt", map_location=device) # Uses GPU since in this Colab we us

======  3  of  24


...

======  23  of  24


<ipython-input-6-9be98728e089>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("lag-llama.ckpt", map_location=device) # Uses GPU since in this Colab we us

======  24  of  24


<ipython-input-6-9be98728e089>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("lag-llama.ckpt", map_location=device) # Uses GPU since in this Colab we us

,forecasting_method_name,root,to_be_predicted,forecasted,success
0,lag-llama,"[0, 0, 0, 0, 0, 0, 0]",[0],[0],1
1,lag-llama,"[0, 0, 0, 0, 0, 0, 0]",[1],[0],0
2,lag-llama,"[0, 0, 0, 0, 1, 0, 0]",[0],[0],1
3,lag-llama,"[0, 0, 0, 0, 0, 1, 0, 0]",[0],[0],1
4,lag-llama,"[0, 0, 0, 0, 0, 0, 1, 1]",[0],[0],1
5,lag-llama,"[0, 0, 0, 0, 0, 0, 0, 1]",[0],[0],1
6,lag-llama,"[0, 0, 0, 0, 0, 0, 0, 1]",[1],[0],0
7,lag-llama,"[0, 0, 0, 1, 1, 0, 0, 0]",[0],[0],1
8,lag-llama,"[0, 0, 1, 0, 0, 0, 0, 0]",[1],[0],0
9,lag-llama,"[0, 1, 0, 1, 0, 1, 0, 1]",[0],[0],1


In [ ]:
res.to_csv("bin_seq_last_digit_pred_lag_llama.csv", index=False)

In [ ]:
c1, c2, c3, c4 = create_original_length_test_pool()

In [ ]:
g_df = create_results_by_list_percentages(c4, 4, percentages_list)
g_df

======  1  of  30


<ipython-input-6-9be98728e089>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("lag-llama.ckpt", map_location=device) # Uses GPU since in this Colab we us

======  2  of  30


<ipython-input-6-9be98728e089>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("lag-llama.ckpt", map_location=device) # Uses GPU since in this Colab we us

======  3  of  30


...

======  29  of  30


<ipython-input-6-9be98728e089>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("lag-llama.ckpt", map_location=device) # Uses GPU since in this Colab we us

======  30  of  30


<ipython-input-6-9be98728e089>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("lag-llama.ckpt", map_location=device) # Uses GPU since in this Colab we us

,complexity,forecasting_method_name,root_10,to_predict_10,forecasted_10,sort_simi_percen_10,gral_simi_percen_10,complexity,forecasting_method_name,root_25,...,sort_simi_percen_50,gral_simi_percen_50,complexity,forecasting_method_name,root_75,to_predict_75,forecasted_75,sort_simi_percen_75,gral_simi_percen_75,seq
0,4,lag-llama,"[29, 57, 68, 120, 134, 140, 173, 197, 283]",[313],[185],0.0,0.0,4,lag-llama,"[29, 57, 68, 120, 134, 140, 173, 197]",...,0.0,0.0,4,lag-llama,"[29, 57, 68]","[120, 134, 140, 173, 197, 283, 313]","[63, 71, -3, 64, 50, 14, 26]",0.000000,0.000000,"29, 57, 68, 120, 134, 140, 173, 197, 283, 313"
1,4,lag-llama,"[24, 26, 36, 40, 184, 226, 244, 384, 391]",[423],[377],0.0,0.0,4,lag-llama,"[24, 26, 36, 40, 184, 226, 244, 384]",...,0.0,0.0,4,lag-llama,"[24, 26, 36]","[40, 184, 226, 244, 384, 391, 423]","[28, 24, 20, 32, 37, 20, 36]",0.000000,0.000000,"24, 26, 36, 40, 184, 226, 244, 384, 391, 423"
2,4,lag-llama,"[90, 203, 212, 235, 270, 324, 342, 352, 371]",[417],[409],0.0,0.0,4,lag-llama,"[90, 203, 212, 235, 270, 324, 342, 352]",...,0.0,0.0,4,lag-llama,"[90, 203, 212]","[235, 270, 324, 342, 352, 371, 417]","[255, 276, 190, 417, 320, 229, 6]",0.000000,7.692308,"90, 203, 212, 235, 270, 324, 342, 352, 371, 417"
3,4,lag-llama,"[20, 48, 95, 234, 282, 296, 352, 402, 428]",[481],[400],0.0,0.0,4,lag-llama,"[20, 48, 95, 234, 282, 296, 352, 402]",...,0.0,0.0,4,lag-llama,"[20, 48, 95]","[234, 282, 296, 352, 402, 428, 481]","[98, 47, 68, 53, 75, 21, 62]",0.000000,0.000000,"20, 48, 95, 234, 282, 296, 352, 402, 428, 481"
4,4,lag-llama,"[62, 98, 130, 154, 290, 315, 324, 385, 408]",[447],[367],0.0,0.0,4,lag-llama,"[62, 98, 130, 154, 290, 315, 324, 385]",...,0.0,0.0,4,lag-llama,"[62, 98, 130]","[154, 290, 315, 324, 385, 408, 447]","[95, 90, 71, 87, 85, 82, 83]",0.000000,0.000000,"62, 98, 130, 154, 290, 315, 324, 385, 408, 447"
5,4,lag-llama,"[2, 42, 66, 102, 153, 195, 201, 252, 306]",[396],[264],0.0,0.0,4,lag-llama,"[2, 42, 66, 102, 153, 195, 201, 252]",...,0.0,0.0,4,lag-llama,"[2, 42, 66]","[102, 153, 195, 201, 252, 306, 396]","[41, -54, -6, 36, 54, 122, 50]",0.000000,0.000000,"2, 42, 66, 102, 153, 195, 201, 252, 306, 396"
6,4,lag-llama,"[128, 151, 153, 217, 224, 332, 382, 400, 450]",[478],[351],0.0,0.0,4,lag-llama,"[128, 151, 153, 217, 224, 332, 382, 400]",...,0.0,0.0,4,lag-llama,"[128, 151, 153]","[217, 224, 332, 382, 400, 450, 478]","[199, 135, 192, 286, 181, 78, 160]",0.000000,0.000000,"128, 151, 153, 217, 224, 332, 382, 400, 450, 478"
7,4,lag-llama,"[26, 50, 114, 148, 160, 170, 274, 347, 432]",[497],[260],0.0,0.0,4,lag-llama,"[26, 50, 114, 148, 160, 170, 274, 347]",...,0.0,0.0,4,lag-llama,"[26, 50, 114]","[148, 160, 170, 274, 347, 432, 497]","[89, 16, 102, 75, 89, 58, 53]",0.000000,0.000000,"26, 50, 114, 148, 160, 170, 274, 347, 432, 497"
8,4,lag-llama,"[48, 94, 176, 177, 219, 276, 282, 283, 459]",[488],[291],0.0,0.0,4,lag-llama,"[48, 94, 176, 177, 219, 276, 282, 283]",...,0.0,0.0,4,lag-llama,"[48, 94, 176]","[177, 219, 276, 282, 283, 459, 488]","[177, 51, 73, 97, 94, 41, 73]",14.285714,8.333333,"48, 94, 176, 177, 219, 276, 282, 283, 459, 488"
9,4,lag-llama,"[139, 252, 272, 281, 304, 361, 370, 415, 438]",[500],[434],0.0,0.0,4,lag-llama,"[139, 252, 272, 281, 304, 361, 370, 415]",...,0.0,0.0,4,lag-llama,"[139, 252, 272]","[281, 304, 361, 370, 415, 438, 500]","[397, 280, 331, 341, 252, 167, 283]",0.000000,0.000000,"139, 252, 272, 281, 304, 361, 370, 415, 438, 500"
